<div style="padding-right: 2rem; padding-bottom: 2rem; width: 100px; height: 100px; align: left;">
<img src="https://sales.captor.se/captor_logo_sv_1600_icketransparent.png" alt="Captor Fund Management AB"/>
</div>
<div>
  <h3>An introduction to the <a href="https://pypi.org/project/openseries/">openseries</a> Python package</h3>
</div>

First we set it all up

In [ ]:
import datetime as dt
import requests
from plotly.offline import init_notebook_mode, iplot

from openseries import (
    OpenTimeSeries,
    OpenFrame,
    ReturnSimulation,
    ValueType,
    efficient_frontier,
    prepare_plot_data,
    sharpeplot,
)

seed = 55
init_notebook_mode()

We define a function to fetch data from an external API

In [ ]:
def get_fund_nav(isin_code: str) -> OpenTimeSeries:

    response = requests.get(url="https://api.qipple.com/public/api/nav", timeout=30)

    if response.status_code // 100 != 2:
        raise ValueError(f"{response.status_code}, {response.text}")

    data = {}
    result = response.json()
    for res in result:
        if res["isin"] == isin_code:
            data.update(res)

    if len(data) == 0:
        raise ValueError(
            f"Request for NAV series using ISIN {isin_code} returned no data."

        )

    return OpenTimeSeries.from_arrays(
        name=data["longName"],
        timeseries_id=data["_id"],
        baseccy=data["currency"],
        dates=data["dates"],
        values=data["navPerUnit"],
        valuetype=ValueType.PRICE,
    )

Here we fetch NAV (price per unit) for some of the Captor mutual funds

In [ ]:
basket = OpenFrame(
    [
        get_fund_nav(isin_code="SE0015243886"),
        get_fund_nav(isin_code="SE0011337195"),
        get_fund_nav(isin_code="SE0011670843"),
        get_fund_nav(isin_code="SE0017832280"),
        get_fund_nav(isin_code="SE0017832330"),
    ]
)
basket = basket.value_nan_handle().trunc_frame().to_cumret()

We then make a portfolio of the funds based on a weight strategy

In [ ]:
portfolio = OpenTimeSeries.from_df(basket.make_portfolio(name="Portfolio", weight_strat="eq_weights"))
basket = basket.add_timeseries(portfolio)

And create and display a Plotly plot of all the data

In [ ]:
figure, _ = basket.plot_series(tick_fmt=".1%", auto_open=False, output_type="div", add_logo=False)
figure = figure.update_layout(
    height=600, 
    font_size=10, 
    legend=dict(
        yanchor="bottom", 
        y=-0.3, 
        xanchor="right", 
        x=0.98, 
        orientation="h"
    )
)
iplot(figure, link_text="")

Here we calculate and display a Pandas Dataframe with some analyses

In [ ]:
df = basket.all_properties(properties=[
        "arithmetic_ret",
        "vol",
        "ret_vol_ratio",
        "sortino_ratio",
        "worst_month",
        "cvar_down",
        "first_indices",
        "last_indices",
    ]
)

In [ ]:
df.columns = df.columns.droplevel(level=1)

formats = [
        "{:.2%}",
        "{:.2%}",
        "{:.2f}",
        "{:.2f}",
        "{:.2%}",
        "{:.2%}",
        "{:%Y-%m-%d}",
        "{:%Y-%m-%d}",
    ]

for item, f in zip(df.index, formats):
    df.loc[item] = df.loc[item].apply(
        lambda x: x if isinstance(x, str) else f.format(x)
    )
df

Below we illustrate the portfolio simulation tools in the package. We first simulate 4 assets.

In [ ]:
series = ReturnSimulation.from_merton_jump_gbm(
    number_of_sims=4,
    trading_days=2512,
    mean_annual_return=0.05,
    mean_annual_vol=0.1,
    jumps_lamda=0.1,
    jumps_sigma=0.3,
    jumps_mu=-0.2,
    trading_days_in_year=252,
    seed=seed,
)
assets = OpenFrame(
    [
        OpenTimeSeries.from_df(
            series.to_dataframe(name="Asset", end=dt.date(2023, 12, 29)),
            column_nmbr=serie,
        )
        for serie in range(series.number_of_sims)
    ],
).to_cumret()

We then simulate portfolio weights and find the efficient frontier and optimal portfolio using return over volatility as the target measure.

In [ ]:
simulations = 5000
points = 30

current = OpenTimeSeries.from_df(
    assets.make_portfolio(
        name="Current Portfolio",
        weight_strat="eq_weights",
    ),
)
frontier, simulated, optimum = efficient_frontier(
    eframe=assets,
    num_ports=simulations,
    seed=seed,
    frontier_points=points,
)
plotframe = prepare_plot_data(
    assets=assets,
    current=current,
    optimized=optimum,
)
simfigure, _ = sharpeplot(
    sim_frame=simulated,
    line_frame=frontier,
    point_frame=plotframe,
    point_frame_mode="markers+text",
    title=False,
    add_logo=False,
    auto_open=False,
    output_type="div",
)
simfigure = simfigure.update_layout(font_size=10, height=600)
iplot(simfigure, link_text="")